In [33]:
from sklearn.datasets import load_digits
digits = load_digits()

In [34]:
print(digits.data.shape)
print(digits.target[44])

(1797, 64)
7


In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

In [36]:
data_tensor = torch.Tensor(digits.data)
target_tensor = F.one_hot(torch.Tensor(digits.target).long(), 10).float()

dataset = TensorDataset(data_tensor, target_tensor)

# Create dataloader
# Wait we are not doing backpropogation so cant we just make batch_size the entire thing?
batch_size = 1797
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Simple FFN approach

class FFN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        linear_relu_stack = nn.Sequential(
            nn.Linear(8*8, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
            # nn.ReLU(),
            # nn.Linear(512, 10),
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = linear_relu_stack(x)
        return logits
        return F.softmax(logits, dim=0)

model = FFN()

epochs = 100
batch_size = 10

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for i in range(epochs):
    for j in range(0, len(data_tensor), batch_size):
        pred = model(data_tensor[j:j+batch_size])
        
        target = target_tensor[j:j+batch_size]

        loss = loss_fn(pred, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print(loss.item())

# Double check
print(torch.argmax(model(data_tensor[5])))
print(target_tensor[5])